In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
print("Started")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

df = pd.read_csv('train.csv')
df = df.fillna('')
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

df['question1'] = df['question1'].apply(preprocess_text)
df['question2'] = df['question2'].apply(preprocess_text)

# Combine questions for vectorization
df['combined'] = df['question1'] + ' ' + df['question2']

Started


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1076)>


In [3]:
from scipy.sparse import hstack
X_train, X_test, y_train, y_test = train_test_split(df[['question1', 'question2']], df['is_duplicate'], test_size=0.2, random_state=42)

vectorizer_q1 = TfidfVectorizer()
vectorizer_q2 = TfidfVectorizer()

X_train_q1_tfidf = vectorizer_q1.fit_transform(X_train['question1'])
X_train_q2_tfidf = vectorizer_q2.fit_transform(X_train['question2'])

X_test_q1_tfidf = vectorizer_q1.transform(X_test['question1'])
X_test_q2_tfidf = vectorizer_q2.transform(X_test['question2'])

X_train_tfidf = hstack([X_train_q1_tfidf, X_train_q2_tfidf])
X_test_tfidf = hstack([X_test_q1_tfidf, X_test_q2_tfidf])

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

import joblib
joblib.dump(model, 'question_similarity_model.pkl')
joblib.dump(vectorizer_q1, 'tfidf_vectorizer_q1.pkl')
joblib.dump(vectorizer_q2, 'tfidf_vectorizer_q2.pkl')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.77      0.87      0.81     50803
           1       0.71      0.55      0.62     30055

    accuracy                           0.75     80858
   macro avg       0.74      0.71      0.72     80858
weighted avg       0.75      0.75      0.74     80858

Accuracy: 0.7502658982413614


['tfidf_vectorizer_q2.pkl']

In [11]:
from tqdm import tqdm
class RandomForestWithProgress(RandomForestClassifier):
    def fit(self, X, y):
        self.n_estimators_ = self.n_estimators
        for i in tqdm(range(self.n_estimators)):
            super(RandomForestWithProgress, self).fit(X, y)
        return self

rf_model = RandomForestWithProgress(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
y_pred_rf = rf_model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred_rf))


              precision    recall  f1-score   support

           0       0.77      0.87      0.81     50803
           1       0.74      0.59      0.68     30055

    accuracy                           0.79     80858
   macro avg       0.74      0.71      0.72     80858
weighted avg       0.75      0.75      0.74     80858

Accuracy: 0.790265898241361
    


In [12]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_tfidf, y_train)
y_pred_xgb = xgb_model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred_xgb))


              precision    recall  f1-score   support

           0       0.77      0.87      0.81     50803
           1       0.71      0.55      0.62     30055

    accuracy                           0.77     80858
   macro avg       0.74      0.71      0.72     80858
weighted avg       0.75      0.75      0.74     80858

Accuracy: 0.770000000000000
    
